In [1]:
# imports
import sys
import openmm
import openmm.app as app
import openmm.unit as u

import numpy as np
import mdtraj
import torch

import rdkit.Chem as Chem
import rdkit.Chem.AllChem as AllChem
from rdkit.Geometry import Point3D

from parmed.charmm import CharmmPsfFile, CharmmCrdFile, CharmmParameterSet
from parmed.openmm import StateDataReporter

In [25]:
pdb = app.pdbfile.PDBFile('1uao_8_autopsf.pdb')

forcefield = app.forcefield.ForceField('amber14/protein.ff14SBonlysc.xml', 'implicit/gbn2.xml')
system = forcefield.createSystem(
    pdb.topology, nonbondedMethod = app.forcefield.NoCutoff, constraints = app.forcefield.HBonds,
)

integrator = openmm.VerletIntegrator(0.002 * u.picoseconds)
platform = openmm.Platform.getPlatformByName("CUDA")

# start at 1, since we assume 0 is being used by PyTorch, to avoid GPU memory issues
assigned_gpu = np.random.randint(1, torch.cuda.device_count())
prop = dict(CudaPrecision="mixed", DeviceIndex=f"{assigned_gpu}")
simulation = app.Simulation(pdb.topology, system, integrator, platform, prop)

In [26]:
def optimize():
    simulation.minimizeEnergy(maxIterations=1000)
    return simulation.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(u.kilocalories_per_mole)

In [29]:
pdb = app.pdbfile.PDBFile('1uao_3_autopsf.pdb')
simulation.context.setPositions(pdb.positions)
optimize()

Quantity(value=-393.31267179497576, unit=kilocalorie/mole)

In [13]:
def np_to_mm(arr: np.ndarray, unit: openmm.unit=u.angstrom):
    wrapped_val = openmm.unit.quantity.Quantity(arr, unit)
    return wrapped_val

In [17]:
pdb = "/home/yppatel/misc/clean_idp_rl/disordered_mol_untrained/mol0.pdb"
mol = Chem.rdmolfiles.MolFromPDBFile(pdb, removeHs=False)
AllChem.EmbedMultipleConfs(mol, numConfs=1000, numThreads=-1)

[14:40:04] Molecule does not have explicit Hs. Consider calling AddHs()


In [18]:
energies = []
for i in range(mol.GetNumConformers()):
    pos_conf = np_to_mm(mol.GetConformer(i).GetPositions())
    simulation.context.setPositions(pos_conf)
    energies.append(optimize())

In [19]:
print(sorted(energies))

[Quantity(value=-365.79530733181986, unit=kilocalorie/mole), Quantity(value=-362.1552793939543, unit=kilocalorie/mole), Quantity(value=-362.07884275797454, unit=kilocalorie/mole), Quantity(value=-361.7243338826107, unit=kilocalorie/mole), Quantity(value=-361.53078931972004, unit=kilocalorie/mole), Quantity(value=-359.17140212877115, unit=kilocalorie/mole), Quantity(value=-359.0626295471738, unit=kilocalorie/mole), Quantity(value=-358.84752249711863, unit=kilocalorie/mole), Quantity(value=-357.6355060513033, unit=kilocalorie/mole), Quantity(value=-357.52852110005125, unit=kilocalorie/mole), Quantity(value=-357.3724987858036, unit=kilocalorie/mole), Quantity(value=-357.05444752585544, unit=kilocalorie/mole), Quantity(value=-356.9178481434095, unit=kilocalorie/mole), Quantity(value=-356.7328320705867, unit=kilocalorie/mole), Quantity(value=-356.03122854396076, unit=kilocalorie/mole), Quantity(value=-355.8595735333723, unit=kilocalorie/mole), Quantity(value=-355.61441629204245, unit=kiloca